In [138]:
# Closure example 01
def foo():
    a = 5
    def bar():
        print(a)
    return bar

func = foo()
func()

5


In [ ]:
# Closure example 02
x = 25
def foo():
    def bar():
        print(x)
    return bar

# del(x) --> It deletes the x variable and the function call func() = bar() can't find x anymore.
func = foo()
del(x)
func()

'''
NameError                                 Traceback (most recent call last)
Cell In[18], line 12
     10 func = foo()
     11 del(x)
---> 12 func()

Cell In[18], line 5, in foo.<locals>.bar()
      4 def bar():
----> 5     print(x)

NameError: name 'x' is not defined
'''

In [139]:
# Closure example 03
x = 25
def foo(value):
    def bar():
        print(value)
    return bar

func = foo(x)
del(x)
func()

25


In [140]:
# Closure example 04
x = 105
def foo(value):
    def bar():
        print(value)
    return bar
x = foo(x)
x()

105


In [141]:
# Closure example 05
def parent(arg1, arg2):
    value = 44
    my_dict = {'a':'Chocolate', 'b': 50}
    def child():
        print(arg1, arg2)
        print(value)
        print(my_dict)
    
    return child

func = parent(10, 100)
func()
print([cell.cell_contents for cell in func.__closure__])

10 100
44
{'a': 'Chocolate', 'b': 50}
[10, 100, {'a': 'Chocolate', 'b': 50}, 44]


In [143]:
print(func.__closure__)
print(type(func.__closure__))
print(len(func.__closure__))
print(type(func.__closure__[0]))

(<cell at 0x107c6feb0: int object at 0x105ef5710>, <cell at 0x107c6f790: int object at 0x105ef6250>, <cell at 0x107c6e9e0: dict object at 0x311fd21c0>, <cell at 0x3120d3940: int object at 0x105ef5b50>)
<class 'tuple'>
4
<class 'cell'>


In [150]:
# Decorator example 01

def multiply(a, b):
    return a * b

def double_args(func):
    def wrapper(a, b):
        return func(a * 2, b * 2)
    return wrapper


new_multiply = double_args(multiply)
print(new_multiply(2, 5))

@double_args
def a_multiply(a, b):
    return a * b + 2

def print_args(func):
    def wrapper(*args):
        print(args)
        print(f'{func.__name__} is called with arguments: {args}')
        # return func(*args)
    return wrapper

print('testing ... {}'.format(a_multiply(2, 5)))

b_multiply = print_args(multiply)
print('testing ... {}'.format(b_multiply(3, 6)))

# print(dir(a_multiply))

print(a_multiply.__name__)
print(a_multiply.__doc__)

40
testing ... 42
(3, 6)
multiply is called with arguments: (3, 6)
testing ... None
wrapper
None


In [151]:
import inspect

def multiply(a, b):
    return a * b

def print_args(func):
    def wrapper(*args):
        signature = inspect.signature(func)
        parameters = signature.parameters
        #print(parameters)
        params_string = ', '.join([f'{name}={value}' for name, value in zip(parameters.keys(), args)])
        print(f'{func.__name__} is called with arguments: ' + params_string)
        return func(*args)
    return wrapper

multiply = print_args(multiply)
print('testing ... {}'.format(multiply(3, 6)))

multiply is called with arguments: a=3, b=6
testing ... 18


In [152]:
import inspect

def my_function(param1, param2, param3="default"):
    pass

signature = inspect.signature(my_function)
print(signature)
parameters = signature.parameters
print(parameters)

a = [f'{name}={param.default}' for name, param in parameters.items()]
print(a)

(param1, param2, param3='default')
OrderedDict({'param1': <Parameter "param1">, 'param2': <Parameter "param2">, 'param3': <Parameter "param3='default'">})
["param1=<class 'inspect._empty'>", "param2=<class 'inspect._empty'>", 'param3=default']


In [153]:
import time

def timer(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        print(f'{func.__name__}: {end_time - start_time}')
        return result
    return wrapper
              
def memoize(func):
    cache = {}
    def wrapper(n):
        if n not in cache:
            cache[n] = func(n)
        return cache[n]
    return wrapper


def general_memoize(func):
    cache = {}
    def wrapper(*args, **kwargs):
        kwargs_key = tuple(sorted(kwargs.items()))
        if (args, kwargs_key) not in cache:
            cache[(args, kwargs_key)] = func(*args, **kwargs)
        return cache[(args, kwargs_key)]
    return wrapper


def counter(func):
    def wrapper(*args, **kwargs):
        wrapper.count += 1
        # Call the function being decorated and return the result
        return func(*args, **kwargs)
    # Set count to 0 to initialize call count for each new decorated function
    wrapper.count = 0
    # Return the new decorated function
    return wrapper
    
@counter
@memoize
def calculate(n):
    if n == 1:
        return 1
    elif n % 2 == 1:
        return calculate(3 * n + 1) + 1
    else:
        return calculate(n // 2) + 1

@timer
def run():
    result = [calculate(i) for i in range(1, 100001)]
    print(max(result))

run()

print('calculate is called {} times'.format(calculate.count))


# @general_memoize
# def slow_function(a, b):
#     print('Sleeping ...')
#     time.sleep(5)
#     return a + b

# print(slow_function(1, 2))
# print(slow_function(1, 2))


351
run: 0.13143181800842285
calculate is called 317211 times


In [157]:

print(calculate(1))
print(calculate(2))
print(calculate(3))
print(calculate(4))
print(calculate(8))


1
2
8
3
4


# Witout memoize function
351
run: 3.2158761024475098
calculate is called 10853840 times


# With memoize function
351
run: 0.13365602493286133
calculate is called 317211 times



In [ ]:
# Return return type
def print_return_type(func):
  # Define wrapper(), the decorated function
  def wrapper(*args, **kwargs):
    # Call the function being decorated
    result = func(*args, **kwargs)
    print('{}() returned type {}'.format(
      func.__name__, type(result)
    ))
    return result
  # Return the decorated function
  return wrapper
  
@print_return_type
def foo(value):
  return value
  
print(foo(42))
print(foo([1, 2, 3]))
print(foo({'a': 42}))

In [ ]:
# Counting function calls
from functools import wraps

def counter(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        wrapper.count += 1
        # Call the function being decorated and return the result
        return func(*args, **kwargs)
    
    # Set count to 0 to initialize call count for each new decorated function
    wrapper.count = 0
    # Return the new decorated function
    return wrapper

# Decorate foo() with the counter() decorator
@counter
def foo():
    ''' a foo function. '''
    print('calling foo()')
  
foo()
foo()

print('foo() was called {} times.'.format(foo.count))

In [ ]:
foo()
print('foo() was called {} times.'.format(foo.count))

In [ ]:
foo.__doc__

In [ ]:
foo.__wrapped__()

In [ ]:
foo()
print('foo() was called {} times.'.format(foo.count))

# Decorator Factory. Function that returns the decorator.

In [ ]:
def run_n_times(n):
    def decorator(func):
        def wrapper(*args, **kwargs):
            for i in range(1, n+1):
                print(f'{i}', end=' ')
                func(*args, **kwargs)
        return wrapper
    return decorator

@run_n_times(3)
def print_sum(a, b):
    print(a + b)

print_sum(10, 100)

@run_n_times(6)
def print_hello():
    print('hello')

print_hello()

In [122]:
def html(open_tag, close_tag):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
          msg = func(*args, **kwargs)
          return '{}{}{}'.format(open_tag, msg, close_tag)
        # Return the decorated function
        return wrapper
    # Return the decorator
    return decorator

# Make hello() return bolded text
@html('<b>', '</b>')
def hello(name):
  return 'Hello {}!'.format(name)
  
print(hello('Alice'))

@html('<i>', '</i>')
def goodbye(name):
    return 'Goodbye {}'.format(name)

print(goodbye('Alice'))

# Wrap the result of hello_goodbye() in <div> and </div>
@html('<div>', '</div>')
def hello_goodbye(name):
  return '\n{}\n{}\n'.format(hello(name), goodbye(name))
  
print(hello_goodbye('Alice'))

<b>Hello Alice!</b>
<i>Goodbye Alice</i>
<div>
<b>Hello Alice!</b>
<i>Goodbye Alice</i>
</div>


# Timeout

In [ ]:
import time
import signal
import functools

def raise_timeout(*args, **kwargs):
    raise TimeoutError()

signal.signal(signalnum=signal.SIGALRM, handler=raise_timeout)
signal.alarm(5)

def timeout_in_5s(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        signal.alarm(5)
        try:
            return func(*args, **kwargs)
        finally:
            signal.alarm(0)
    return wrapper

@timeout_in_5s
def foo():
    time.sleep(5)
    print('foo')



In [127]:
import time
import functools
import signal

def raise_timeout(*args, **kwargs):
    raise TimeoutError()

signal.signal(signalnum=signal.SIGALRM, handler=raise_timeout)

def timeout(n_seconds):
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            signal.alarm(n_seconds)
            try:
                return func(*args, **kwargs)
            finally:
                signal.alarm(0)
        return wrapper
    return decorator

@timeout(5)
def foo():
    time.sleep(1)
    print('foo')

foo()

foo


In [128]:
# tagging a function
def tag(*tags):
    # Define a new decorator, named "decorator", to return
    def decorator(func):
        # Ensure the decorated function keeps its metadata
        @wraps(func)
        def wrapper(*args, **kwargs):
            # Call the function being decorated and return the result
            return func(*args, **kwargs)
        wrapper.tags = tags
        return wrapper
    # Return the new decorator
    return decorator

@tag('test', 'this is a tag')
def foo():
    pass

print(foo.tags)

('test', 'this is a tag')


In [137]:
# Test if the function is returning the correct data type

def returns(return_type):
  # Complete the returns() decorator
  def decorator(func):
    def wrapper(*args, **kwargs):
      result = func(*args, **kwargs)
      assert type(result) == return_type
      return result
    return wrapper
  return decorator
  
@returns(dict)
def foo(value):
  return value

try:
  print(foo([1,2,3]))
except AssertionError:
  print('foo() did not return a dict!')
  

foo() did not return a dict!


In [129]:
a = {1: 10, 2: 20}
type(a)

dict

In [136]:
a = [1, 10, 2, 20]
print(type(a))
print(*a)

<class 'list'>
1 10 2 20


In [131]:
type(a) == list

True

In [133]:
a = (1, 10, 2, 20)

In [135]:
print(*a)

1 10 2 20
